---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
def answer_one():
    return df[df['Gold'] == df['Gold'].max()].index.values[0]
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [4]:
def answer_two():
    df['Difference'] = df['Gold'] - df['Gold.1']
    return df[df['Difference'] == df['Difference'].max()].index.values[0]
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [5]:
def answer_three():
    filtered_df = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    filtered_df['Summer_Gold_Ratio'] = (filtered_df['Gold']-filtered_df['Gold.1'])/(filtered_df['Gold.2'])
    return filtered_df[filtered_df['Summer_Gold_Ratio'] == filtered_df['Summer_Gold_Ratio'].max()].index.values[0]
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [6]:
def answer_four():
    points_dict = {}
    for country in df.index.values:
        points_dict[country] = df.loc[country][-6]*3 + df.loc[country][-5]*2 + df.loc[country][-4]*1
    Points = pd.Series(points_dict)
    return Points
answer_four()

Afghanistan                  2
Algeria                     27
Argentina                  130
Armenia                     16
Australasia                 22
Australia                  923
Austria                    569
Azerbaijan                  43
Bahamas                     24
Bahrain                      1
Barbados                     1
Belarus                    154
Belgium                    276
Bermuda                      1
Bohemia                      5
Botswana                     2
Brazil                     184
British West Indies          2
Bulgaria                   411
Burundi                      3
Cameroon                    12
Canada                     846
Chile                       24
China                     1120
Chinese Taipei              32
Colombia                    29
Costa Rica                   7
Croatia                     67
Cuba                       420
Cyprus                       2
                          ... 
Spain                      268
Sri Lank

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [7]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [8]:
census_df.shape

(3193, 100)

In [9]:
census_df.SUMLEV.value_counts()

50    3142
40      51
Name: SUMLEV, dtype: int64

In [10]:
census_df[census_df['STATE'] == 34]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1804,40,1,2,34,0,New Jersey,New Jersey,8791894,8791936,8803881,...,-5.099617,-5.563775,-5.137475,-6.216888,-7.292230,0.457420,-0.353542,0.470243,0.180654,-0.913121
1805,50,1,2,34,1,New Jersey,Atlantic County,274549,274549,274761,...,-7.293044,-6.488261,-5.995464,-11.202928,-13.523297,-1.571369,-1.147337,-0.326037,-4.758798,-7.010867
1806,50,1,2,34,3,New Jersey,Bergen County,905116,905117,906895,...,-0.717223,-1.199548,-0.450633,-2.428030,-2.800130,5.414866,4.565916,5.832237,4.694694,4.247219
1807,50,1,2,34,5,New Jersey,Burlington County,448734,448728,449218,...,-0.744637,-2.996310,-6.168662,-2.182553,-5.072688,1.153632,-0.512322,-3.894245,0.357468,-2.397594
1808,50,1,2,34,7,New Jersey,Camden County,513657,513678,513594,...,-7.536188,-7.452898,-8.240683,-8.301976,-6.783509,-4.686500,-4.533327,-5.223848,-4.712144,-3.184296
1809,50,1,2,34,9,New Jersey,Cape May County,97265,97265,97263,...,-4.446508,-1.377924,-3.472962,-2.405166,-4.860958,-2.176829,1.232880,-1.206179,0.115030,-2.114832
1810,50,1,2,34,11,New Jersey,Cumberland County,156898,156898,157155,...,-5.166415,-6.718541,-10.818232,-5.094308,-13.829553,-2.058940,-4.269668,-8.250332,-1.903998,-10.583059
1811,50,1,2,34,13,New Jersey,Essex County,783969,783987,784597,...,-10.033068,-11.485267,-7.989145,-8.330932,-8.901052,-2.965125,-5.167926,-1.157957,-0.570024,-1.191915
1812,50,1,2,34,15,New Jersey,Gloucester County,288288,288288,288701,...,-1.110398,-2.724499,-1.856318,1.019247,-1.648417,-0.100316,-1.747271,-0.797044,2.262315,-0.405236
1813,50,1,2,34,17,New Jersey,Hudson County,634266,634277,636317,...,-6.484171,-9.597070,-11.381356,-16.819860,-16.396174,8.397507,4.004015,3.188416,-0.406581,-0.160674


In [11]:
census_df[census_df['SUMLEV'] == 50].STNAME.value_counts().idxmax()

'Texas'

In [12]:
def answer_five():
    return census_df[census_df['SUMLEV'] == 50].STNAME.value_counts().idxmax()
answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [47]:
def answer_six():
    pop_dict = {}
    filtered_census = census_df[census_df['SUMLEV']==50]
    filtered_census = filtered_census[['STNAME', 'CTYNAME', 'CENSUS2010POP']]
    for state in filtered_census.STNAME.unique():
        pop_dict[state] = filtered_census[filtered_census['STNAME'] == state].nlargest(3,['CENSUS2010POP']).CENSUS2010POP.sum(axis=0)
    state_pop = pd.Series(pop_dict)
    return [state for state in state_pop.nlargest(3).index]
answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [90]:
def answer_seven():
    estimate_dic = {}
    census_estimate = census_df[census_df['SUMLEV']==50]
    census_estimate['NEWNAME'] = census_estimate['STNAME'] + '-' + census_estimate['CTYNAME']
    census_estimate = census_estimate[['NEWNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']]
    census_estimate = census_estimate.set_index('NEWNAME')
    for name in census_estimate.index.values:
        minimum = census_estimate.loc[name].min()
        maximum = census_estimate.loc[name].max()
        difference = abs(maximum-minimum)
        estimate_dic[name] = difference 
    estimate_series = pd.Series(estimate_dic)
    return estimate_series.nlargest(1).index[0].split('-')[1]
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [11]:
def answer_eight():
    return "YOUR ANSWER HERE"